# **Testing the Simulator**

In [1]:
import numpy as np
import sys

sys.path.append("../tools")
import mc

In [2]:
import mc_sampler

In [3]:
P = np.array([[0, 0.75, 0.25],
              [0.5, 0, 0.5],
              [0.75, 0.25, 0]])

## **Testing the sampler**

In [7]:
#Test case 1:
#1 state, initial state is 0
#carrying out multiple samplings, that must represent [0, 0.75, 0.25] vector

results = [0, 0, 0]
state = [0]
N_SAMPLES = 10000
for i in range(N_SAMPLES):
    result = mc_sampler.sample_chain(P, state)[0]
    results[result] = results[result]+1
    
print("{:.2f}, {:.2f}, {:.2f}".format(results[0]/N_SAMPLES, results[1]/N_SAMPLES, results[2]/N_SAMPLES))

0.00, 0.75, 0.25


In [8]:
#Test case 2:
#2 states, initial state is 0, 1
#carrying out multiple samplings, that must represent [0, 0.75, 0.25], [0.5, 0, 0.5] vectors

results = [[0, 0, 0], [0, 0, 0]]
state = [0, 1]
N_SAMPLES = 10000
for i in range(N_SAMPLES):
    result = mc_sampler.sample_chain(P, state)
    for r in range(2):
        results[r][result[r]] = results[r][result[r]]+1
    
print("{:.2f}, {:.2f}, {:.2f}".format(results[0][0]/N_SAMPLES, results[0][1]/N_SAMPLES, results[0][2]/N_SAMPLES))
print("{:.2f}, {:.2f}, {:.2f}".format(results[1][0]/N_SAMPLES, results[1][1]/N_SAMPLES, results[1][2]/N_SAMPLES))

0.00, 0.76, 0.24
0.50, 0.00, 0.50


In [7]:
#Test case 3
#Long run test
#starting from the initial state, making huge amount of steps
#resulting vector shall represent the stationary distribution

π = mc.calculate_stationary_distribution(P)

results = [0, 0, 0]
state = [0]
N_SAMPLES = int(1e6)
for i in range(N_SAMPLES):
    result = mc_sampler.sample_chain(P, state)[0]
    state = [result]
    results[result] = results[result]+1
    
print("expected\t\t", π)
print("test result\t\t{:.2f}, {:.2f}, {:.2f}".format(results[0]/N_SAMPLES, results[1]/N_SAMPLES, results[2]/N_SAMPLES))

expected		 [0.37837838 0.35135135 0.27027027]
test result		0.38, 0.35, 0.27
